In [1]:
# Устанавливает библиотеку для работы с БД
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'D:\ProgramFiles\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
# Загрузка библиотек
import pandas as pd
import numpy as np
import pyodbc
import re
from scipy.stats import mode

# Функции

In [83]:
# Удаление лишних символов в колонке с номиналами
def clean_nom(x): 
    if x == None:
        return x
    x = re.sub('(\\r\\n)*(\/\*)+[\w\d\s=-]*(\*\/)+(\\r\\n)*', '\n', x)
    x = re.sub('\r', '', x)
    x = re.sub('\t', ' ', x)
    x = re.sub('  ', ' ', x)
    x = x.split('\n')
    x = [i for i in x if i.strip() != ""]
    x = [i.rstrip() for i in x]
    x = [i.lstrip() for i in x]
    x = [i.strip() for i in x]
    return x

# Записывает номиналы в словарь
def get_nom_df(x):
    if x !=None:
        for row in x:
            try:
                name, val = row.split(' ')
                if nom_dict.get(name, 0)==0:
                    nom_dict[name] = [float(val)]
                else:
                    nom_dict[name].append(float(val))
            except ValueError:
                print(row)
    return 

# Запрос из БД и сохранение данных в таблице

In [84]:
# Параметры для входа в БД
conn_str = (
    r'DRIVER={SQL Server};'
    r'SERVER=192.168.0.250\SQLEXPRESS;'
    r'DATABASE=db1;'
    r'UID=NordLab;'
    r'PWD=nordnord;')

In [85]:
# Текст запроса
query = '''
        Select    *  from Channel_States
        Full outer join Devices on Devices.Device_ID=Channel_States.Device_ID
        Full outer join Orders on Orders.Order_ID=Devices.Order_Number
        Full outer join Device_Types on Device_Types.Device_Type_ID=Devices.Device_Type
        Full outer join Sensors on Sensors.Sensor_ID =Channel_States.Sensor_ID 
        Full outer join Sensor_Type on Sensor_Type.Sensor_Type_ID = Sensors.Sensor_Type
        '''

In [86]:
cnxn = pyodbc.connect(conn_str) # Подключение к БД
DF = pd.read_sql_query(query, cnxn) # Запрос к БД и сохранение данных в формате pd.DataFrame

In [87]:
DF.columns

Index(['Channel_State_ID', 'Device_ID', 'Sensor_ID', 'SPL_ID', 'Calibr_SPL_ID',
       'Calibr_File', 'Calibr_Data', 'Sensor_Tr_func', 'NOM', 'Date',
       'Iteration_Number', 'Channel_Type', 'Comment', 'Self_Calib_ID',
       'Device_ID', 'Device_Type', 'Device_SN', 'Comment', 'Date', 'In_Work',
       'Z_Locked', 'X_locked', 'Y_Locked', 'Order_Number', 'Sensitivity',
       'Order_ID', 'Number', 'Comment', 'Customer', 'Date', 'In_Work', 'Term',
       'Amount', 'Responsible', 'Device_Type_ID', 'Device_Type_Name',
       'Comment', 'Date', 'Sensitivity', 'Type', 'Default_Calib_File',
       'Default_Calib_SPL', 'Default_Z_Sensor_Type', 'Default_X_Sensor_Type',
       'Default_Y_Sensor_Type', 'Sensor_ID', 'Sensor_Type', 'Sensor_Number',
       'Date', 'Comment', 'FilledBy', 'Sensor_Type_ID', 'Sensor_Type_Name',
       'Date', 'Comment'],
      dtype='object')

# Статистика номиналов

In [88]:
# Список всех типов приборов
DF.Device_Type_Name.unique()

array(['CME-4211 BH', 'MTSS-2003', 'MTSS-1001', 'MTSS-1043A', 'MTSS-1011',
       'CME-6011', 'CME-4211', 'MTSS-1003', None, 'MTSS-1031А',
       'MTSS-1021', 'MTSS-1033A', 'MTSS-1043A DIG', 'CME-4311LT',
       'CME-3211', 'MTSS-1041A', 'CME-4311', 'CME-3011h', 'METR-01',
       'CME-6111', 'CME-6011A', 'METR-03', 'MTSS-1031H', 'MTSS-1031G',
       'Imp', 'UMG_hydro', 'Gen_I', 'UMG_Geo', 'NS_hydro', 'METR-01 MHD'],
      dtype=object)

In [89]:
# Список каналов
DF.Channel_Type.unique()

array(['V', None, 'S', 'Z', 'X', 'Y'], dtype=object)

In [193]:
device_type = 'CME-4211' # тип прибора
channel = 'X' # номер канала
min_iter = 0 # минимальное число итераций
year = 2020 # минимальный год для среза

In [191]:
col_list = ['Device_Type_Name', 'Device_SN', 'NOM', 'Iteration_Number', 'Channel_Type', 'Date'] # необходимые колонки для статистики
new_df = DF[col_list] # срез таблицы
new_df= new_df.loc[:,~new_df.columns.duplicated()] # удаление повторов
new_df.NOM = new_df.NOM.apply(lambda x: clean_nom(x)) # преобразование колонки с номиналами
new_df.Date = pd.DatetimeIndex(new_df.Date) # преобразование даты в нужный формат
new_df['Year'] = new_df.Date.dt.year #  добавление колонки год

In [194]:
# Сохраняет последние итерации каждого прибора
sel_df = new_df[(new_df.Device_Type_Name == device_type) &
                (new_df.Channel_Type == channel) &
                (new_df.Iteration_Number >= min_iter) &
                (new_df.Year >= year)]\
        .sort_values('Iteration_Number')\
        .groupby("Device_SN").tail(1)
sel_df

,Device_Type_Name,Device_SN,NOM,Iteration_Number,Channel_Type,Date,Year
7831,CME-4211,W_183_42R,"[R1 0.0124, R3 0.011, R5 0.047, R7 0.33, R8 0....",3.0,X,2021-03-01 13:19:26.660,2021.0
31006,CME-4211,Temp_Li_3_42,"[R1 0.0018, R3 0.015, R5 0.047, R7 0.160, R8 0...",4.0,X,2021-01-11 16:55:34.930,2021.0
30872,CME-4211,W_NT4_42_2.7,"[R1 0.00068, Rt1 0.0022, R3 0.008, R5 0.047, R...",4.0,X,2021-05-31 18:31:31.390,2021.0
30963,CME-4211,Temp_Li_1_42,"[R1 0.0018, R3 0.015, R5 0.047, R7 0.160, R8 0...",4.0,X,2021-01-11 16:54:52.393,2021.0
30981,CME-4211,Temp_Li_2_42,"[R1 0.0018, R3 0.015, R5 0.047, R7 0.160, R8 0...",4.0,X,2021-01-11 16:55:13.517,2021.0
32026,CME-4211,Temp_Li_6_42,"[R1 0.0018, R3 0.015, R5 0.047, R7 0.160, R8 0...",5.0,X,2021-01-15 16:30:19.340,2021.0
32047,CME-4211,W_163_42R,"[R1 0.0064, R3 0.015, R5 0.047, R7 0.33, R8 0....",5.0,X,2021-01-21 15:14:04.907,2021.0
32017,CME-4211,Temp_Li_5_42,"[R1 0.0018, R3 0.015, R5 0.047, R7 0.160, R8 0...",5.0,X,2021-01-15 16:29:53.150,2021.0
32002,CME-4211,Temp_Li_4_42,"[R1 0.0018, R3 0.015, R5 0.047, R7 0.160, R8 0...",5.0,X,2021-01-15 16:29:15.190,2021.0
3916,CME-4211,W_162_42R,"[R1 0.0062, R3 0.015, R5 0.047, R7 0.33, R8 0....",5.0,X,2020-12-03 12:30:22.960,2020.0


In [195]:
# Записывает список номиналов в словарь, печатает строки, которые не были добавлены
nom_dict = {}
sel_df.NOM.apply(lambda x: get_nom_df(x))
print()

In [196]:
# Подсчет статистики по каждому номиналу
col = ['Median', 'Mode', 'Mode_count', 'Mean', 'Max', 'Min',  'Count']
idx = nom_dict.keys()
stat_table = pd.DataFrame(columns=col, index = idx)
for key in nom_dict.keys():
    stat_table['Mean'][key] = np.round(np.asarray(nom_dict[key]).mean(),4)
    stat_table['Max'][key] = np.asarray(nom_dict[key]).max()
    stat_table['Min'][key] = np.asarray(nom_dict[key]).min()
    stat_table['Median'][key] = np.median(np.asarray(nom_dict[key]))
    stat_table['Mode'][key] = mode(np.asarray(nom_dict[key]))[0][0]
    stat_table['Mode_count'][key] = mode(np.asarray(nom_dict[key]))[1][0]
    stat_table['Count'][key] = len(nom_dict[key])

In [197]:
stat_table

,Median,Mode,Mode_count,Mean,Max,Min,Count
R1,0.00555,0.0018,10,0.0094,0.07,0.00068,36
R3,0.015,0.015,28,0.0138,0.015,0.008,36
R5,0.047,0.047,36,0.047,0.047,0.047,36
R7,0.16,0.16,26,0.2072,0.33,0.16,36
R8,0.016,0.016,23,0.0221,0.033,0.016,36
R9,0.11,0.11,24,0.1424,0.22,0.11,34
R10,0.75,0.75,24,0.9256,1.5,0.16,36
R11,5.1,5.1,26,6.4611,10.0,5.1,36
R12,0.075,0.075,24,0.0948,0.15,0.056,36
R13,0.039,0.039,24,0.0516,0.082,0.039,34


In [17]:
stat_table.to_csv('stat_MTSS-1031А.csv')